In [1]:
import pandas as pd

In [2]:
gamesdata = pd.read_csv(r'C:\Users\redye\Desktop\NBACrit\games.csv')
recorddata = pd.read_csv(r'C:\Users\redye\Desktop\NBA Exist\Betting-Model-NBA-master\data\ranking.csv')

In [3]:
# Adds the total points scored in a game to the end of the dataframe
# This will be our y value
point_total = gamesdata.PTS_home + gamesdata.PTS_away
gamesdata["point_total"] = point_total

In [4]:
# Making a dataframe to fill in values for the first games of the 2018 season
gamesdata2017 = gamesdata.loc[gamesdata.SEASON >= 2016]

In [5]:
# Removing entries before 2018 season
gamesdata = gamesdata.loc[gamesdata.SEASON >= 2017]
gamesdata.sort_values(by=["GAME_DATE_EST"])
gamesdata = gamesdata.reset_index()

In [6]:
# Removing unwanted features
features_games = [
    "GAME_DATE_EST",
    "HOME_TEAM_ID",
    "VISITOR_TEAM_ID",
    "SEASON",
    "PTS_home",
    "PTS_away",
    "point_total",
]

In [7]:
gamesdata2017 = gamesdata2017[features_games].copy()
gamesdata = gamesdata[features_games].copy()

In [8]:
features_record = [
    "TEAM_ID",
    "SEASON_ID",
    "STANDINGSDATE",
    "G",
    "W",
    "L",
    "W_PCT",
    "HOME_RECORD",
    "ROAD_RECORD",
]

In [9]:
recorddata = recorddata[features_record].copy()

In [10]:
# Fixing recorddata season column
recorddata["SEASON_ID"] = recorddata["SEASON_ID"].apply(lambda x: x - 20000)

In [11]:
# Removing entries before 2018 season from recorddata
recorddata = recorddata.loc[recorddata.SEASON_ID >= 2018]
recorddata.sort_values(by=["SEASON_ID"])
recorddata = recorddata.reset_index()

In [12]:
# Adding mean points scored for the home team in the past 10 games
# If games played this season < 10, go to last season data
avgppglast5_home = []  # ppg - points per game
avgpalast5_home = []  # pa - points againts
avgppglast5_away = []  # ppg - points per game
avgpalast5_away = []  # pa - points againts

In [13]:
for ind in range(len(gamesdata)):
    date = gamesdata["GAME_DATE_EST"][ind]
    hteam = gamesdata["HOME_TEAM_ID"][ind]
    # When they are the home team
    match_stats = gamesdata.loc[
        (gamesdata.GAME_DATE_EST < date) & (gamesdata.HOME_TEAM_ID == hteam)
    ]
    # When they are the visitor team
    match_stats_away = gamesdata.loc[
        (gamesdata.GAME_DATE_EST < date) & (gamesdata.VISITOR_TEAM_ID == hteam)
    ]
    # match_stats - list of all previous games (datafrome)

    # First I check to see if there are no recorded games (first game of season)
    if not match_stats.empty:
        prevgamescount = len(match_stats.index)  # How many games have been played
    if match_stats.empty or prevgamescount < 5:
        # If there are less than 5, then go back to the previous season and take the average
        lastseas = gamesdata2017.loc[
            (gamesdata2017.GAME_DATE_EST < date) & (gamesdata2017.HOME_TEAM_ID == hteam)
        ]
        avgppglast10val = lastseas.PTS_home.mean(axis=0)
        avgpalast10val = lastseas.PTS_away.mean(axis=0)
    else:
        only10 = match_stats.iloc[:10]
        avgppglast10val = only10.PTS_home.mean(axis=0)
        avgpalast10val = only10.PTS_away.mean(axis=0)

    avgppglast5_home.append(avgppglast10val)
    avgpalast5_home.append(avgpalast10val)

    # Do the same thing but for when they are the away team
    if not match_stats_away.empty:
        prevgamescounta = len(match_stats_away.index)
    if match_stats_away.empty or prevgamescounta < 5:
        lastseasa = gamesdata2017.loc[
            (gamesdata2017.GAME_DATE_EST < date)
            & (gamesdata2017.VISITOR_TEAM_ID == hteam)
        ]
        avgppglast10vala = lastseasa.PTS_away.mean(axis=0)
        avgpalast10vala = lastseasa.PTS_home.mean(axis=0)
    else:
        only10 = match_stats_away.iloc[:10]
        avgppglast10vala = only10.PTS_away.mean(axis=0)
        avgpalast10vala = only10.PTS_home.mean(axis=0)

    avgppglast5_away.append(avgppglast10vala)
    avgpalast5_away.append(avgpalast10vala)

    # avgppglast10val = statistics.mean(prevgamelist)

In [14]:
gamesdata["avgppglast5_at_home"] = avgppglast5_home
gamesdata["avgpalast5_at_home"] = avgpalast5_home
gamesdata["avgppglast5_at_away"] = avgppglast5_away
gamesdata["avgpalast5_at_away"] = avgpalast5_away

In [15]:
# Taking the average of the last 5 away games and 5 home games
gamesdata["point_average_last10"] = (
    gamesdata["avgppglast5_at_home"] + gamesdata["avgppglast5_at_away"]
) / 2
gamesdata["point_againts_average_last10"] = (
    gamesdata["avgpalast5_at_home"] + gamesdata["avgpalast5_at_away"]
) / 2

In [16]:
# Same as previous but now we do the away team in the matchup
aavgppglast5_home = []  # ppg - points per game
aavgpalast5_home = []  # pa - points againts
aavgppglast5_away = []  # ppg - points per game
aavgpalast5_away = []  # pa - points againts

In [17]:
for ind in range(len(gamesdata)):
    date = gamesdata["GAME_DATE_EST"][ind]
    hteam = gamesdata["VISITOR_TEAM_ID"][ind]

    match_stats = gamesdata.loc[
        (gamesdata.GAME_DATE_EST < date) & (gamesdata.HOME_TEAM_ID == hteam)
    ]

    match_stats_away = gamesdata.loc[
        (gamesdata.GAME_DATE_EST < date) & (gamesdata.VISITOR_TEAM_ID == hteam)
    ]

    if not match_stats.empty:
        prevgamescount = len(match_stats.index)
    if match_stats.empty or prevgamescount < 5:
        lastseas = gamesdata2017.loc[
            (gamesdata2017.GAME_DATE_EST < date) & (gamesdata2017.HOME_TEAM_ID == hteam)
        ]
        avgppglast10val = lastseas.PTS_home.mean(axis=0)
        avgpalast10val = lastseas.PTS_away.mean(axis=0)
    else:
        only10 = match_stats.iloc[:10]
        avgppglast10val = only10.PTS_home.mean(axis=0)
        avgpalast10val = only10.PTS_away.mean(axis=0)

    aavgppglast5_home.append(avgppglast10val)
    aavgpalast5_home.append(avgpalast10val)

    if not match_stats_away.empty:
        prevgamescounta = len(match_stats_away.index)
    if match_stats_away.empty or prevgamescounta < 5:
        lastseasa = gamesdata2017.loc[
            (gamesdata2017.GAME_DATE_EST < date)
            & (gamesdata2017.VISITOR_TEAM_ID == hteam)
        ]
        avgppglast10vala = lastseasa.PTS_away.mean(axis=0)
        avgpalast10vala = lastseasa.PTS_home.mean(axis=0)
    else:
        only10 = match_stats_away.iloc[:10]
        avgppglast10vala = only10.PTS_away.mean(axis=0)
        avgpalast10vala = only10.PTS_home.mean(axis=0)
    aavgppglast5_away.append(avgppglast10vala)
    aavgpalast5_away.append(avgpalast10vala)


In [18]:
gamesdata["aavgppglast5_at_home"] = aavgppglast5_home
gamesdata["aavgpalast5_at_home"] = aavgpalast5_home
gamesdata["aavgppglast5_at_away"] = aavgppglast5_away
gamesdata["aavgpalast5_at_away"] = aavgpalast5_away

In [19]:
gamesdata["away_point_average_last10"] = (
    gamesdata["aavgppglast5_at_home"] + gamesdata["aavgppglast5_at_away"]
) / 2
gamesdata["away_point_againts_average_last10"] = (
    gamesdata["aavgpalast5_at_home"] + gamesdata["aavgpalast5_at_away"]
) / 2

In [20]:
# drop all the columns that were used to take 5 home - 5 away
gamesdata = gamesdata.drop(
    [
        "aavgppglast5_at_home",
        "aavgpalast5_at_home",
        "aavgppglast5_at_away",
        "aavgpalast5_at_away",
        "avgppglast5_at_home",
        "avgpalast5_at_home",
        "avgppglast5_at_away",
        "avgpalast5_at_away",
    ],
    axis=1,
)

In [21]:
# Now add the current ranking of each team as featues
g = []
w = []
l = []
wpc = []
homerec = []
roadrec = []

In [22]:
for ind in range(
    len(gamesdata)
):  # for every entry in games data, it matches the previously reported stats for the team, season, and date before current.
    date = gamesdata["GAME_DATE_EST"][ind]
    seas = gamesdata["SEASON"][ind]
    hteam = gamesdata["HOME_TEAM_ID"][ind]
    temp1 = recorddata.loc[
        (recorddata.STANDINGSDATE < date)
        & (recorddata.SEASON_ID == seas)
        & (recorddata.TEAM_ID == hteam)
    ]
    if temp1.empty:  # first game of the season check
        temp2 = 0
    else:  # not the first game of the season
        temp2 = temp1.iloc[0]["G"]
        temp3 = temp1.iloc[0]["W"]
        temp4 = temp1.iloc[0]["L"]
        temp5 = temp1.iloc[0]["W_PCT"]
        temp6 = temp1.iloc[0]["HOME_RECORD"]
        temp7 = temp1.iloc[0]["ROAD_RECORD"]
    g.append(
        temp2
    )  # appends all the wanted values to lists in order to put into the main dataset
    w.append(temp3)
    l.append(temp4)
    wpc.append(temp5)
    homerec.append(temp6)
    roadrec.append(temp7)

In [23]:
# data is already matched in order of row, so just add the list as a column
gamesdata["cgp"] = g
gamesdata["wins"] = w
gamesdata["losses"] = l
gamesdata["winpercent"] = wpc
gamesdata["homerecord"] = homerec
gamesdata["awayrecord"] = roadrec

In [24]:
# Same as previous for the away team in the matchup
g1 = []
w1 = []
l1 = []
wpc1 = []
homerec1 = []
roadrec1 = []
ht_hw = []
ht_aw = []
at_hw = []
at_aw = []

In [25]:
for ind in range(len(gamesdata)):
    date = gamesdata["GAME_DATE_EST"][ind]
    seas = gamesdata["SEASON"][ind]
    ateam = gamesdata["VISITOR_TEAM_ID"][ind]
    temp1 = recorddata.loc[
        (recorddata.STANDINGSDATE < date)
        & (recorddata.SEASON_ID == seas)
        & (recorddata.TEAM_ID == ateam)
    ]
    if temp1.empty:
        temp2 = 0
    else:
        temp2 = temp1.iloc[0]["G"]
        temp3 = temp1.iloc[0]["W"]
        temp4 = temp1.iloc[0]["L"]
        temp5 = temp1.iloc[0]["W_PCT"]
        temp6 = temp1.iloc[0]["HOME_RECORD"]
        temp7 = temp1.iloc[0]["ROAD_RECORD"]
    g1.append(temp2)
    w1.append(temp3)
    l1.append(temp4)
    wpc1.append(temp5)
    homerec1.append(temp6)
    roadrec1.append(temp7)

In [26]:
# data is already matched in order of row, so just add the list as a column
gamesdata["cgp_away"] = g1
gamesdata["wins_away"] = w1
gamesdata["losses_away"] = l1
gamesdata["winpercen_away"] = wpc1
gamesdata["homerecord_away"] = homerec1
gamesdata["awayrecord_away"] = roadrec1

In [27]:
# Splitting up the record column so that it is not a string, but rather 2 columns of ints
for ind in range(len(gamesdata)):
    hh1 = gamesdata["homerecord"][ind]
    hh2 = hh1.split("-")
    ht_hw.append(hh2[0])

    ha1 = gamesdata["awayrecord"][ind]
    ha2 = ha1.split("-")
    ht_aw.append(ha2[0])

    ah1 = gamesdata["homerecord_away"][ind]
    ah2 = ah1.split("-")
    at_hw.append(ah2[0])

    aa1 = gamesdata["awayrecord_away"][ind]
    aa2 = aa1.split("-")
    at_aw.append(aa2[0])

In [28]:
# Adding the new feature lists as columns
gamesdata["hometeam-homewins"] = ht_hw
gamesdata["hometeam-awaywins"] = ht_aw
gamesdata["awayteam-homewins"] = at_hw
gamesdata["awayteam-awaywins"] = at_aw

In [29]:
# Since theses columns were made from a string, they need to be converted object -> int
gamesdata["hometeam-homewins"] = gamesdata["hometeam-homewins"].astype(str).astype(int)
gamesdata["hometeam-awaywins"] = gamesdata["hometeam-awaywins"].astype(str).astype(int)
gamesdata["awayteam-homewins"] = gamesdata["awayteam-homewins"].astype(str).astype(int)
gamesdata["awayteam-awaywins"] = gamesdata["awayteam-awaywins"].astype(str).astype(int)

In [30]:
# Drop old record columns
gamesdata = gamesdata.drop(
    columns=["homerecord", "awayrecord", "homerecord_away", "awayrecord_away"]
)

In [31]:
gamesdata = gamesdata.drop(
    columns=["cgp", "wins", "losses", "winpercent", "cgp_away", "wins_away", "losses_away", "winpercen_away", "hometeam-homewins", "hometeam-awaywins", "awayteam-homewins", "awayteam-awaywins"]
)

In [32]:
mask = (gamesdata['GAME_DATE_EST'] >= '2017-09-30') & (gamesdata['GAME_DATE_EST'] <= '2018-06-08')
gamesdata = gamesdata.loc[mask]
gamesdata

,GAME_DATE_EST,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,PTS_away,point_total,point_average_last10,point_againts_average_last10,away_point_average_last10,away_point_againts_average_last10
2343,2018-06-08,1610612739,1610612744,2017,85.0,108.0,193.0,102.500000,103.150000,108.850000,102.550000
2344,2018-06-06,1610612739,1610612744,2017,102.0,110.0,212.0,102.300000,103.150000,109.200000,102.450000
2345,2018-06-03,1610612744,1610612739,2017,122.0,103.0,225.0,108.750000,101.900000,101.650000,101.650000
2346,2018-05-31,1610612744,1610612739,2017,124.0,114.0,238.0,108.550000,102.500000,102.100000,100.900000
2347,2018-05-28,1610612745,1610612744,2017,92.0,101.0,193.0,103.250000,102.750000,108.800000,104.200000
...,...,...,...,...,...,...,...,...,...,...,...
3720,2017-10-02,1610612747,1610612743,2017,107.0,113.0,220.0,104.352683,111.290244,111.172238,110.442829
3721,2017-10-01,1610612748,1610612737,2017,96.0,90.0,186.0,103.215909,101.812747,102.953014,103.322695
3722,2017-10-01,1610612761,1610612746,2017,121.0,113.0,234.0,105.528750,102.570833,106.798980,103.888963
3723,2017-09-30,1610612744,1610612743,2017,102.0,108.0,210.0,116.226415,104.320755,111.176893,110.534142


In [34]:
gamesdata.to_csv(r'C:\Users\redye\Desktop\NBACrit\last_10_test2.csv', index=False)